# Warm Start from a Completed Hyper-Parameter Tuning Job
Once the previous hyper-parameter tuning job completes, we can perform another round of optimization using `Warm Start`.

Warm start configuration allows you to create a new tuning job with the learning gathered in a parent tuning job by specifying up to 5 parent tuning jobs. If a warm start configuration is specified, Automatic Model Tuning will load the previous [hyperparameter set, objective metrics values] to warm start the new tuning job. This means, you can continue optimizing your model from the point you finished your previous tuning job experiment.

<img src="img/hpt-warmstart.png" width="90%" align="left">

In [1]:
!pip install -q sagemaker-experiments==0.1.13

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Pre-Requisite

## The previous hyper-parameter tuning job needs to complete, before we can perform another round of optimization using `Warm Start`.

In [3]:
%store -r tuning_job_name

In [4]:
try:
    tuning_job_name
    print('+++++++++++++++++++++++++++++++++++++++++++++')
    print('OK. Previous Tuning Job has completed.')
    print('+++++++++++++++++++++++++++++++++++++++++++++')
except NameError:
    print('+++++++++++++++++++++++++++++++++++++++++++++')
    print('STOP: Please run the previous Hyperparameter Tuning notebook.')
    print('+++++++++++++++++++++++++++++++++++++++++++++')

+++++++++++++++++++++++++++++++++++++++++++++
OK. Previous Tuning Job has completed.
+++++++++++++++++++++++++++++++++++++++++++++


In [5]:
print(tuning_job_name)

tensorflow-training-200822-1959


### Check the status of the previous Hyperparameter Job

In [6]:
job_description = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

In [7]:
if not bool(job_description):
    print('STOP: Please run the previous Hyperparameter Tuning notebook before you continue.')
elif job_description['HyperParameterTuningJobStatus'] == 'Completed':
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('OK. Previous Tuning Job has completed. Please continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('+++++++++++++++++++++++++++++++++++++++++++++')
    print('STOP: Please run the previous Hyperparameter Tuning notebook.')
    print('+++++++++++++++++++++++++++++++++++++++++++++')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
OK. Previous Tuning Job has completed. Please continue.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Specify the S3 Location of the Features

In [8]:
%store -r processed_train_data_s3_uri

In [9]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-train


In [10]:
%store -r processed_validation_data_s3_uri

In [11]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-validation


In [12]:
%store -r processed_test_data_s3_uri

In [13]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-test


In [14]:
print(processed_train_data_s3_uri)
!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-train
2020-08-22 18:38:14      10636 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-08-22 18:38:14      11676 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [15]:
print(processed_validation_data_s3_uri)
!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-validation
2020-08-22 18:38:15        657 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-08-22 18:38:15        644 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [16]:
print(processed_test_data_s3_uri)
!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-test
2020-08-22 18:38:15        716 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-08-22 18:38:15        650 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [17]:
s3_input_train_data = sagemaker.s3_input(s3_data=processed_train_data_s3_uri, 
                                         distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=processed_validation_data_s3_uri, 
                                              distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=processed_test_data_s3_uri, 
                                        distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/sagemaker-scikit-learn-2020-08-22-18-33-57-088/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


In [18]:
!cat src/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.8.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.

# Setup Hyper-Parameters for Classification Layer
First, retrieve `max_seq_length` from the prepare phase.

In [19]:
%store -r max_seq_length

In [20]:
print(max_seq_length)

64


In [21]:
epochs=1
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=50
validation_steps=50
test_steps=50
train_instance_count=1
train_instance_type='ml.c5.4xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=True
enable_sagemaker_debugger=False
enable_checkpointing=False
enable_tensorboard=False
input_mode='Pipe'
run_validation=True
run_test=True
run_sample_predictions=True

# Track the Optimizations Within our Experiment

In [22]:
%store -r experiment_name

In [23]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1598124929


In [24]:
%store -r trial_name

In [25]:
print(trial_name)

trial-1598124929


In [26]:
import time
from smexperiments.trial import Trial

timestamp = '{}'.format(int(time.time()))

trial = Trial.load(trial_name=trial_name)
print(trial)

Trial(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fd0bf246048>,trial_name='trial-1598124929',trial_arn='arn:aws:sagemaker:us-west-2:250107111215:experiment-trial/trial-1598124929',display_name='trial-1598124929',experiment_name='Amazon-Customer-Reviews-BERT-Experiment-1598124929',creation_time=datetime.datetime(2020, 8, 22, 19, 35, 29, 651000, tzinfo=tzlocal()),created_by={},last_modified_time=datetime.datetime(2020, 8, 22, 20, 23, 51, 375000, tzinfo=tzlocal()),last_modified_by={},response_metadata={'RequestId': '1d8a204d-3b50-4634-b4fd-a080581aa41e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1d8a204d-3b50-4634-b4fd-a080581aa41e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '326', 'date': 'Sat, 22 Aug 2020 21:13:02 GMT'}, 'RetryAttempts': 0})


In [27]:
from smexperiments.tracker import Tracker

tracker_optimize = Tracker.create(display_name='optimize-2', 
                                  sagemaker_boto_client=sm)

optimize_trial_component_name = tracker_optimize.trial_component.trial_component_name
print('Optimize trial component name {}'.format(optimize_trial_component_name))

Optimize trial component name TrialComponent-2020-08-22-211302-fqxj


# Attach the `deploy` Trial Component and Tracker as a Component to the Trial

In [28]:
trial.add_trial_component(tracker_optimize.trial_component)

# Setup Dynamic Hyper-Parameter Ranges to Explore
While not necessary, we can choose to statically define any hyper-parameters that we are not choosing to explore in this WarmStart optimization run.


In [29]:
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import CategoricalParameter
from sagemaker.tuner import HyperparameterTuner
                                                
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.00015, 0.00075, scaling_type='Linear'),
}

# Track the Hyper-Parameter Ranges

In [30]:
tracker_optimize.log_parameters(hyperparameter_ranges)

# must save after logging
tracker_optimize.trial_component.save()

TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fd0a256c1d0>,trial_component_name='TrialComponent-2020-08-22-211302-fqxj',display_name='optimize-2',trial_component_arn='arn:aws:sagemaker:us-west-2:250107111215:experiment-trial-component/trialcomponent-2020-08-22-211302-fqxj',response_metadata={'RequestId': 'b703c0fb-3e79-49de-92fd-0810bc6d6faf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b703c0fb-3e79-49de-92fd-0810bc6d6faf', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Sat, 22 Aug 2020 21:13:03 GMT'}, 'RetryAttempts': 0},parameters={'learning_rate': <sagemaker.parameter.ContinuousParameter object at 0x7fd0bee639b0>},input_artifacts={},output_artifacts={})

# Setup Metrics

In [31]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

In [32]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       train_instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       train_instance_type=train_instance_type,
                       train_volume_size=train_volume_size,
                       py_version='py3',
                       framework_version='2.1.0',
                       hyperparameters={'epochs': epochs,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,                                        
                                        'enable_checkpointing': enable_checkpointing,
                                        'enable_tensorboard': enable_tensorboard,                                        
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
#                       train_max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

# Setup Warm Start Config
We configure `WarmStartConfig` using 1 or more  of the previous hyper-parameter tuning job runs called the `parent` jobs - as well as a `WarmStartType`.  The parents must have finished either with one of the following success or failure states: `Completed`, `Stopped`, or `Failed`.

`WarmStartType` is one of the following strategies:

* `IDENTICAL_DATA_AND_ALGORITHM` uses the same input data and algorithm as the parent tuning jobs, but allows a practitioner to explore more hyper-parameter range values.  Upon completion, a tuning job with this strategy will return an additional field, `OverallBestTrainingJob` containing the best model candidate including this tuning job as well as the completed parent tuning jobs.
* `TRANSFER_LEARNING` allows you to transfer the knowledge from previous tuning jobs.  You can use different input dataset and algorithm - as well as everything from the `IDENTICAL_DATA_AND_ALGORITHM` strategy.

_Note:  Recursive parent-child relationships are not supported._

In [33]:
print('Previous Tuning Job Name: {}'.format(tuning_job_name))

Previous Tuning Job Name: tensorflow-training-200822-1959


In [34]:
from sagemaker.tuner import WarmStartConfig
from sagemaker.tuner import WarmStartTypes

warm_start_config = WarmStartConfig(warm_start_type=WarmStartTypes.IDENTICAL_DATA_AND_ALGORITHM, 
                                    parents={tuning_job_name})

# Setup HyperparameterTuner with Warm Start Config including New Hyper-Parameter Ranges

In [35]:
objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_type='Maximize',
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=2,
    max_parallel_jobs=1,
    strategy='Bayesian',
    early_stopping_type='Auto',
    warm_start_config=warm_start_config
)

# Start Tuning Job

In [36]:
tuner.fit({'train': s3_input_train_data, 
           'validation': s3_input_validation_data,
           'test': s3_input_test_data
          }, include_cls_metadata=False)

INFO:sagemaker.tuner:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: tensorflow-training-200822-2113


# If You See an Error, Please Wait for the Hyper-Parameter Tuning Job to Complete from the Previous Notebook

##  Check Tuning Job Status

Re-run this cell to track the status.

In [37]:
from pprint import pprint

tuning_job_name = tuner.latest_tuning_job.job_name

job_description = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = job_description['HyperParameterTuningJobStatus']

print('\n')
print(status)
print('\n')
pprint(job_description)

if status != 'Completed':
    job_count = job_description['TrainingJobStatusCounters']['Completed']
    print('Not yet complete, but {} jobs have completed.'.format(job_count))
    
    if job_description.get('BestTrainingJob', None):
        print("Best candidate:")
        pprint(job_description['BestTrainingJob']['TrainingJobName'])
        pprint(job_description['BestTrainingJob']['FinalHyperParameterTuningJobObjectiveMetric'])
    else:
        print("No training jobs have reported results yet.")    



InProgress


{'CreationTime': datetime.datetime(2020, 8, 22, 21, 13, 6, 472000, tzinfo=tzlocal()),
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:hyper-parameter-tuning-job/tensorflow-training-200822-2113',
 'HyperParameterTuningJobConfig': {'HyperParameterTuningJobObjective': {'MetricName': 'validation:accuracy',
                                                                        'Type': 'Maximize'},
                                   'ParameterRanges': {'CategoricalParameterRanges': [],
                                                       'ContinuousParameterRanges': [{'MaxValue': '0.00075',
                                                                                      'MinValue': '0.00015',
                                                                                      'Name': 'learning_rate',
                                                                                      'ScalingType': 'Linear'}],
                                

In [38]:
from IPython.core.display import display, HTML
    
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/{}">Hyper-Parameter Tuning Job</a></b>'.format(region, tuning_job_name)))

# _Please Wait for the ^^ Tuning Job ^^ to Complete Above_

In [39]:
tuner.wait()

...................................................................................................................................................................................................................................................!


# Show the Tuning Job
### _Note:  This will fail at first.  Please wait about 15-30 seconds and re-run._

In [40]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

hp_results = HyperparameterTuningJobAnalytics(
    sagemaker_session=sess, 
    hyperparameter_tuning_job_name=tuning_job_name
)

df_results = hp_results.dataframe()
df_results.shape

(2, 7)

In [41]:
df_results.sort_values('FinalObjectiveValue', ascending=0)

,learning_rate,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.000694,tensorflow-training-200822-2113-002-05e3c8f5,Completed,0.0,2020-08-22 21:25:08+00:00,2020-08-22 21:32:40+00:00,452.0
1,0.000337,tensorflow-training-200822-2113-001-8e7ba164,Completed,0.0,2020-08-22 21:15:08+00:00,2020-08-22 21:23:02+00:00,474.0


# Show the Overall Best Candidate

In [42]:
df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)

,learning_rate,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.000694,tensorflow-training-200822-2113-002-05e3c8f5,Completed,0.0,2020-08-22 21:25:08+00:00,2020-08-22 21:32:40+00:00,452.0


In [43]:
best_candidate_tuning_job_name = df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)['TrainingJobName']

# Log the Best Hyper-Parameter and Objective Metric in the Experiment

Logging `learning_rate` parameter and `accuracy` metric

In [44]:
best_learning_rate = df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)['learning_rate']
print(best_learning_rate)

0    0.000694
Name: learning_rate, dtype: float64


In [45]:
best_accuracy = df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)['FinalObjectiveValue']
print(best_accuracy)

0    0.0
Name: FinalObjectiveValue, dtype: float64


In [46]:
tracker_optimize.log_parameters({
    'learning_rate': float(best_learning_rate)
})

# must save after logging
tracker_optimize.trial_component.save()

TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fd0a256c1d0>,trial_component_name='TrialComponent-2020-08-22-211302-fqxj',display_name='optimize-2',trial_component_arn='arn:aws:sagemaker:us-west-2:250107111215:experiment-trial-component/trialcomponent-2020-08-22-211302-fqxj',response_metadata={'RequestId': '0736935b-28ed-4e7f-950e-ebe71bc5b92f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0736935b-28ed-4e7f-950e-ebe71bc5b92f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Sat, 22 Aug 2020 21:33:35 GMT'}, 'RetryAttempts': 0},parameters={'learning_rate': 0.0006935010572597231},input_artifacts={},output_artifacts={})

In [47]:
tracker_optimize.log_metric('accuracy', float(best_accuracy))

tracker_optimize.trial_component.save()

ERROR:root:'NoneType' object has no attribute 'write'


TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fd0a256c1d0>,trial_component_name='TrialComponent-2020-08-22-211302-fqxj',display_name='optimize-2',trial_component_arn='arn:aws:sagemaker:us-west-2:250107111215:experiment-trial-component/trialcomponent-2020-08-22-211302-fqxj',response_metadata={'RequestId': '78c18e6c-c730-4ef2-848b-dd2b2f1b2140', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '78c18e6c-c730-4ef2-848b-dd2b2f1b2140', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Sat, 22 Aug 2020 21:33:35 GMT'}, 'RetryAttempts': 0},parameters={'learning_rate': 0.0006935010572597231},input_artifacts={},output_artifacts={})

## _Ignore any ^^ ERROR above ^^. This is OK._

# Show Experiment Analytics

In [48]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=['validation:accuracy'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

(4, 63)

In [49]:
lineage_df

,TrialComponentName,DisplayName,balance_dataset,max_seq_length,test_split_percentage,train_split_percentage,validation_split_percentage,raw_data_s3_uri - MediaType,raw_data_s3_uri - Value,test_data_s3_uri - MediaType,...,train - MediaType,train - Value,validation - MediaType,validation - Value,SageMaker.Checkpoints - MediaType,SageMaker.Checkpoints - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value
0,TrialComponent-2020-08-22-193529-qhjz,prepare,True,64.0,0.05,0.9,0.05,s3/uri,s3://sagemaker-us-west-2-250107111215/amazon-r...,s3/uri,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tensorflow-training-2020-08-22-19-35-37-636-aw...,train,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,s3://sagemaker-us-west-2-250107111215/sagemake...,NaN,s3://sagemaker-us-west-2-250107111215/sagemake...,NaN,s3://sagemaker-us-west-2-250107111215/checkpoi...,NaN,s3://sagemaker-us-west-2-250107111215/,NaN,s3://sagemaker-us-west-2-250107111215/tensorfl...
2,TrialComponent-2020-08-22-195922-xchy,optimize-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TrialComponent-2020-08-22-211302-fqxj,optimize-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pass `tuning_job_name` to the Next Notebook

In [50]:
print(best_candidate_tuning_job_name)

0    tensorflow-training-200822-2113-002-05e3c8f5
Name: TrainingJobName, dtype: object


In [51]:
%store best_candidate_tuning_job_name

Stored 'best_candidate_tuning_job_name' (Series)


In [52]:
%store

Stored variables and their in-db values:
autopilot_endpoint_name                          -> 'automl-dm-ep-22-16-47-12'
balance_dataset                                  -> True
best_candidate_tuning_job_name                   -> 0    tensorflow-training-200822-2113-002-05e3c8f5

comprehend_endpoint_arn                          -> 'arn:aws:comprehend:us-west-2:250107111215:documen
df_dataset_metrics                               ->         entity                   instance         
experiment_name                                  -> 'Amazon-Customer-Reviews-BERT-Experiment-159812492
header_train_s3_uri                              -> 's3://sagemaker-us-west-2-250107111215/data/amazon
max_seq_length                                   -> 64
noheader_train_s3_uri                            -> 's3://sagemaker-us-west-2-250107111215/data/amazon
prepare_trial_component_name                     -> 'TrialComponent-2020-08-22-193529-qhjz'
processed_test_data_s3_uri                       -> 's3://

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>